In [1]:
! pip install pandas
! pip install openpyxl

In [2]:
import pandas as pd

In [3]:
from numpy import *


class MatchRating:
    original_rating_name = "Original"
    avg_player_name = "AVG"
    max_iterations = 1000

    def __init__(self, data_sheet_file, rating_scale_base, rating_scale_divisor, rating_base, tie_in_weight):

        self.rating_scale_base = rating_scale_base
        self.rating_scale_divisor = rating_scale_divisor
        self.ranking_boost = rating_base
        self.tie_in_weight = tie_in_weight
        
        self.players: pd.DataFrame = pd.read_excel(data_sheet_file, sheet_name="Players", index_col="ShortName")
        self.matches: pd.DataFrame = pd.read_excel(data_sheet_file, sheet_name="Matches")
        self.matches["IsTieInMatch"] = False
        
        match_columns = self.matches.columns.to_list()
        self.winner_columns = [column for column in match_columns if column.startswith("Winner")]
        self.loser_columns = [column for column in match_columns if column.startswith("Loser")]

        print('Number of matches loaded:' + str(len(self.matches)))

        # --- Add tie-in matches ---
        cols = self.matches.columns
        first_winner = cols[cols.str.startswith("Winner")].values[0]
        first_loser = cols[cols.str.startswith("Loser")].values[0]
        
        tie_in_rows = []
        for player in self.players.index:
            tie_in_rows.append({first_winner: player, first_loser: MatchRating.avg_player_name, "K": self.tie_in_weight})
            tie_in_rows.append({first_winner: MatchRating.avg_player_name, first_loser: player, "K": self.tie_in_weight})

        tie_in_matches = pd.DataFrame(tie_in_rows)
        tie_in_matches["IsTieInMatch"] = True
        self.matches = pd.concat([self.matches, tie_in_matches], ignore_index=True)

        self.__player_ratings: pd.DataFrame = pd.DataFrame(columns=["Rating"], index=self.players.index)
        self.__player_ratings["Rating"] = 0.0
        
        

    def calculate_ratings(self, step_sizes: list, min_player_match_count: int):

        def apply_ranking_boost(rankings):
            rankings["Rating"] = rankings["Rating"] - rankings["Rating"].loc[MatchRating.avg_player_name] + self.ranking_boost
            return rankings
        
        def remove_excluded_players(rankings):
            excluded_players = self.players[self.players["Excluded"] == True].index
            rankings = rankings.drop(excluded_players, errors="ignore")
            return rankings.drop(columns=["Excluded"])
        
        def add_player_metadata(rankings):
            return rankings.merge(self.players, left_index=True, right_index=True)
        
        def add_match_metadata(rankings):
            
            def get_wins_and_losses(player):
                matches = self.matches[self.matches["IsTieInMatch"] == False]
                wins = matches[self.winner_columns].apply(lambda x: (player in x.values), axis=1).sum()
                losses = matches[self.loser_columns].apply(lambda x: (player in x.values), axis=1).sum()
                return pd.Series([wins, losses], index=["Wins", "Losses"])
            
            rankings[["Wins", "Losses"]] = rankings["ShortName"].apply(get_wins_and_losses)
            return rankings
        
        def filter_min_match_count(rankings):
            return rankings[rankings["Wins"] + rankings["Losses"] >= min_player_match_count]

        def add_opponent_rating(rankings):

            def get_opponent_rating(player):
                matches = self.matches[(self.matches["IsTieInMatch"] == False) & self.matches.apply(lambda x: (player in x.values), axis=1)]
                matches = matches[self.winner_columns + self.loser_columns]

                def get_match_opponent_rating(match):
                    if player in match[self.winner_columns].values:
                        team_columns = self.winner_columns
                        opponent_columns = self.loser_columns
                    else:
                        team_columns = self.loser_columns
                        opponent_columns = self.winner_columns

                    team_players = match[team_columns].dropna().values
                    opponent_players = match[opponent_columns].dropna().values

                    player_team_ranking = rankings.loc[team_players, ["Rating"]].mean().values[0]
                    opponent_team_ranking = rankings.loc[opponent_players, ["Rating"]].mean().values[0]

                    return mean(opponent_team_ranking - player_team_ranking)

                opponent_ratings = matches.apply(get_match_opponent_rating, axis=1)
                return round(opponent_ratings.mean(), 0)
        
            rankings["AVG_Opponent_Rating"] = rankings["ShortName"].apply(get_opponent_rating)
            return rankings
        
        def finalize_rankings(rankings):
            rankings = rankings.sort_values(by="Rating", ascending=False).reset_index(drop=True)
            rankings.index += 1
            rankings["Rating"] = rankings["Rating"].round(0).astype(int)
            return rankings.reset_index()
        

        for step_size in step_sizes:
            rankings = self.__calculate_ratings(step_size)

        rankings["ShortName"] = rankings.index

        
        rankings = apply_ranking_boost(rankings)
        rankings = add_player_metadata(rankings)
        rankings = add_match_metadata(rankings)
        rankings = add_opponent_rating(rankings)
        rankings = remove_excluded_players(rankings)
        rankings = filter_min_match_count(rankings)
        rankings = finalize_rankings(rankings)

        return rankings


        
    
    def __calculate_ratings(self, step_size):
        
        total_match_weights  = self.matches["K"].sum()

        for i in range(MatchRating.max_iterations):
            rating_table = self.__create_rating_table(step_size)
            
            match_score_accuracy = self.__calculate_match_score_accuracy(rating_table)        
            
            player_rating_accuracy = pow(match_score_accuracy.prod(), 1 / total_match_weights)
            accuracy_product = player_rating_accuracy.sort_values(ascending=False)

            best_accuracy = accuracy_product.index[0]
            print(f"Step Size={step_size}, Iteration {i + 1}, Accuracy: {round(accuracy_product.iloc[0], 5)}")

            if best_accuracy == MatchRating.original_rating_name:
                return self.__player_ratings
            
            player, rating_change = best_accuracy            
            self.__player_ratings.loc[player, ["Rating"]] += rating_change
            
        return self.__player_ratings

    def __calculate_match_score_accuracy(self, rating_table: pd.DataFrame):

        match_columns = self.matches.columns.to_list()
        winner_columns = [column for column in match_columns if column.startswith("Winner")]
        loser_columns = [column for column in match_columns if column.startswith("Loser")]

        def apply_match_score_accuracy(match):
            winners = match[winner_columns]
            winners = winners[winners.notna()]
            winner_rating = rating_table.loc[winners].sum()
            
            losers = match[loser_columns]
            losers = losers[losers.notna()]
            loser_rating = rating_table.loc[losers].sum()

            accuracy = pow(winner_rating / (winner_rating + loser_rating), match["K"])

            return accuracy

        return self.matches.apply(apply_match_score_accuracy, axis=1, result_type="expand")

    def __create_rating_table(self, step_size):
        
        ratings = []
        columns = []

        original_rating = self.__player_ratings["Rating"]

        def add_rating(rating, name):
            ratings.append(rating.apply(lambda x: pow(self.rating_scale_base, (x / self.rating_scale_divisor))))
            columns.append(name)

        def add_alternative_rating(player, rating_change):
            alternative_rating = original_rating.copy()
            alternative_rating.loc[player] += rating_change
                
            add_rating(alternative_rating, (player, rating_change))

        add_rating(original_rating, MatchRating.original_rating_name)

        for player in self.players.index:
            add_alternative_rating(player, step_size)
            add_alternative_rating(player, -step_size)
            
        rating_table = pd.concat(ratings, axis=1)
        rating_table.columns = columns

        return rating_table

In [4]:
rating_system = MatchRating(
    # data_sheet_file="Data/Test.xlsx",
    # data_sheet_file="Data/UCH_Data.xlsx",
    data_sheet_file="Data/BPLData.xlsx",
    rating_scale_base=10,
    rating_scale_divisor=400,
    rating_base=1000,
    tie_in_weight=0.5
)

Number of matches loaded:131


In [5]:
rankings = rating_system.calculate_ratings(
    step_sizes=[1024,512,256,128],
    min_player_match_count=0
)
rankings

Step Size=1024, Iteration 1, Accuracy: 0.50929
Step Size=1024, Iteration 2, Accuracy: 0.50962
Step Size=1024, Iteration 3, Accuracy: 0.50962
Step Size=512, Iteration 1, Accuracy: 0.51194
Step Size=512, Iteration 2, Accuracy: 0.52031
Step Size=512, Iteration 3, Accuracy: 0.52298
Step Size=512, Iteration 4, Accuracy: 0.52451
Step Size=512, Iteration 5, Accuracy: 0.52542
Step Size=512, Iteration 6, Accuracy: 0.52582
Step Size=512, Iteration 7, Accuracy: 0.52689
Step Size=512, Iteration 8, Accuracy: 0.52721
Step Size=512, Iteration 9, Accuracy: 0.52738
Step Size=512, Iteration 10, Accuracy: 0.52749
Step Size=512, Iteration 11, Accuracy: 0.52752
Step Size=512, Iteration 12, Accuracy: 0.52752
Step Size=256, Iteration 1, Accuracy: 0.53106
Step Size=256, Iteration 2, Accuracy: 0.53319
Step Size=256, Iteration 3, Accuracy: 0.53423
Step Size=256, Iteration 4, Accuracy: 0.53523
Step Size=256, Iteration 5, Accuracy: 0.53611
Step Size=256, Iteration 6, Accuracy: 0.53695
Step Size=256, Iteration 7, 

,index,Rating,ShortName,Name,Wins,Losses,AVG_Opponent_Rating
0,1,1512,TER,Terragen,15,4,-143.0
1,2,1384,MUR,Murk,16,8,-85.0
2,3,1256,PYR,Pyromaniac,10,5,-51.0
3,4,1256,W,Watermelon,14,6,-105.0
4,5,1128,CG,Captain Gaming,74,56,-59.0
5,6,1128,DAR,Daria,35,22,-7.0
6,7,1000,MOS,Dilopho-Mos!,15,16,49.0
7,8,1000,PRO,Capt. Profuse,14,9,15.0
8,9,1000,TB,Tastiestbooger,10,6,1.0
9,10,1000,JOE,Joe,4,6,26.0


In [6]:
rankings = rating_system.calculate_ratings(
    step_sizes=[64,32,16,8],
    min_player_match_count=0
)
rankings

Step Size=64, Iteration 1, Accuracy: 0.55093
Step Size=64, Iteration 2, Accuracy: 0.55157
Step Size=64, Iteration 3, Accuracy: 0.55174
Step Size=64, Iteration 4, Accuracy: 0.55185
Step Size=64, Iteration 5, Accuracy: 0.55193
Step Size=64, Iteration 6, Accuracy: 0.552
Step Size=64, Iteration 7, Accuracy: 0.55207
Step Size=64, Iteration 8, Accuracy: 0.55213
Step Size=64, Iteration 9, Accuracy: 0.55219
Step Size=64, Iteration 10, Accuracy: 0.55224
Step Size=64, Iteration 11, Accuracy: 0.5523
Step Size=64, Iteration 12, Accuracy: 0.55235
Step Size=64, Iteration 13, Accuracy: 0.5524
Step Size=64, Iteration 14, Accuracy: 0.55245
Step Size=64, Iteration 15, Accuracy: 0.5525
Step Size=64, Iteration 16, Accuracy: 0.55254
Step Size=64, Iteration 17, Accuracy: 0.55259
Step Size=64, Iteration 18, Accuracy: 0.55262
Step Size=64, Iteration 19, Accuracy: 0.55265
Step Size=64, Iteration 20, Accuracy: 0.55268
Step Size=64, Iteration 21, Accuracy: 0.55271
Step Size=64, Iteration 22, Accuracy: 0.55275
St

,index,Rating,ShortName,Name,Wins,Losses,AVG_Opponent_Rating
0,1,1584,TER,Terragen,15,4,-159.0
1,2,1456,MUR,Murk,16,8,-104.0
2,3,1336,W,Watermelon,14,6,-122.0
3,4,1304,PYR,Pyromaniac,10,5,-65.0
4,5,1208,DAR,Daria,35,22,-35.0
5,6,1152,CG,Captain Gaming,74,56,-42.0
6,7,1088,MOS,Dilopho-Mos!,15,16,11.0
7,8,1064,TB,Tastiestbooger,10,6,-14.0
8,9,1016,PRO,Capt. Profuse,14,9,19.0
9,10,960,JOE,Joe,4,6,56.0


In [7]:
rankings = rating_system.calculate_ratings(
    step_sizes=[4,2,1,0.5],
    min_player_match_count=10
)
rankings

Step Size=4, Iteration 1, Accuracy: 0.55363
Step Size=4, Iteration 2, Accuracy: 0.55364
Step Size=4, Iteration 3, Accuracy: 0.55364
Step Size=4, Iteration 4, Accuracy: 0.55364
Step Size=4, Iteration 5, Accuracy: 0.55364
Step Size=4, Iteration 6, Accuracy: 0.55364
Step Size=4, Iteration 7, Accuracy: 0.55364
Step Size=4, Iteration 8, Accuracy: 0.55364
Step Size=4, Iteration 9, Accuracy: 0.55364
Step Size=4, Iteration 10, Accuracy: 0.55364
Step Size=4, Iteration 11, Accuracy: 0.55364
Step Size=4, Iteration 12, Accuracy: 0.55364
Step Size=4, Iteration 13, Accuracy: 0.55364
Step Size=4, Iteration 14, Accuracy: 0.55364
Step Size=4, Iteration 15, Accuracy: 0.55364
Step Size=4, Iteration 16, Accuracy: 0.55364
Step Size=4, Iteration 17, Accuracy: 0.55364
Step Size=4, Iteration 18, Accuracy: 0.55364
Step Size=4, Iteration 19, Accuracy: 0.55364
Step Size=4, Iteration 20, Accuracy: 0.55364
Step Size=4, Iteration 21, Accuracy: 0.55364
Step Size=4, Iteration 22, Accuracy: 0.55364
Step Size=4, Iterat

,index,Rating,ShortName,Name,Wins,Losses,AVG_Opponent_Rating
0,1,1590,TER,Terragen,15,4,-161.0
1,2,1458,MUR,Murk,16,8,-104.0
2,3,1338,W,Watermelon,14,6,-122.0
3,4,1306,PYR,Pyromaniac,10,5,-64.0
4,5,1216,DAR,Daria,35,22,-38.0
5,6,1156,CG,Captain Gaming,74,56,-43.0
6,7,1088,MOS,Dilopho-Mos!,15,16,14.0
7,8,1064,TB,Tastiestbooger,10,6,-13.0
8,9,1017,PRO,Capt. Profuse,14,9,19.0
9,10,958,JOE,Joe,4,6,59.0
